<a href="https://colab.research.google.com/github/fatemehm/Federated-Learning-IDS/blob/main/Federated_Learning_IDS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd